In [73]:
!pip install mlxtend --upgrade

import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [74]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [75]:
items = set()
for col in df:
  arr = np.array(df[col].unique())
  items.update(arr)

items = list(items)
items

['Bread',
 nan,
 'Eggs',
 'Wine',
 'Meat',
 'Pencil',
 'Diaper',
 'Milk',
 'Cheese',
 'Bagel']

## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [76]:
#create an itemset based on the products
itemset = set(items)
# encoding the feature
encoded_vals = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]

{nan: 0,
 'Milk': 0,
 'Bagel': 0,
 'Bread': 1,
 'Wine': 1,
 'Eggs': 1,
 'Meat': 1,
 'Pencil': 1,
 'Diaper': 1,
 'Cheese': 1}

In [77]:
# create new dataframe from the encoded features
df = pd.DataFrame(encoded_vals) 
  # show the new dataframe
df.head()

,NaN,Milk,Bagel,Bread,Wine,Eggs,Meat,Pencil,Diaper,Cheese
0,0,0,0,1,1,1,1,1,1,1
1,0,1,0,1,1,0,1,1,1,1
2,1,1,0,0,1,1,1,0,0,1
3,1,1,0,0,1,1,1,0,0,1
4,1,0,0,0,1,0,1,1,0,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [78]:
df.drop([np.nan], axis=1, inplace=True)
df.head()

,Milk,Bagel,Bread,Wine,Eggs,Meat,Pencil,Diaper,Cheese
0,0,0,1,1,1,1,1,1,1
1,1,0,1,1,0,1,1,1,1
2,1,0,0,1,1,1,0,0,1
3,1,0,0,1,1,1,0,0,1
4,0,0,0,1,0,1,1,0,0


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [82]:
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules

patterns = apriori(df, min_support = 0.2, use_colnames = True)

sets = patterns.sort_values('support', ascending = False)[:10]
sets.head()

/usr/local/lib/python3.7/dist-packages/mlxtend/frequent_patterns/fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


,support,itemsets
2,0.504762,(Bread)
0,0.501587,(Milk)
8,0.501587,(Cheese)
5,0.476190,(Meat)
3,0.438095,(Wine)


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [83]:
rules = association_rules(sets, metric="lift")
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cheese),(Meat),0.501587,0.476190,0.32381,0.64557,1.355696,0.084958,1.477891
1,(Meat),(Cheese),0.476190,0.501587,0.32381,0.68000,1.355696,0.084958,1.557540
